In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
import os

PATH = "book-crossings"
books_filename = os.path.join(PATH, 'BX-Books.csv')
ratings_filename = os.path.join(PATH, 'BX-Book-Ratings.csv')


In [44]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})


In [81]:
# Merge books and ratings to get the titles
df_merged = pd.merge(df_ratings, df_books[['isbn', 'title']], on='isbn')
df_merged.head(5)

,user,isbn,rating,title
0,276725,034545104X,0.0,Flesh Tones: A Novel
1,2313,034545104X,5.0,Flesh Tones: A Novel
2,6543,034545104X,0.0,Flesh Tones: A Novel
3,8680,034545104X,5.0,Flesh Tones: A Novel
4,10314,034545104X,9.0,Flesh Tones: A Novel


In [136]:
# Count ratings per user and per book
user_counts = df_merged['user'].value_counts()
book_counts = df_merged['title'].value_counts()
# user_counts.head(5)
book_counts.head(5)

title
Wild Animus                   2502
The Lovely Bones: A Novel     1295
The Da Vinci Code              897
The Nanny Diaries: A Novel     821
A Painted House                818
Name: count, dtype: int64

In [143]:
# Start with the ratings dataframe
df = df_ratings

# Step 1: Filter out users and books with too few ratings
user_counts = df['user'].value_counts()
book_counts = df['isbn'].value_counts()

# Filter to keep users with at least 200 ratings
popular_users = user_counts[user_counts >= 200].index
df = df[df['user'].isin(popular_users)]

# Filter to keep books with at least 100 ratings
popular_books = book_counts[book_counts >= 100].index
df = df[df['isbn'].isin(popular_books)]

# Step 2: Merge in book titles from df_books
df = pd.merge(right=df, left=df_books, on="isbn")

# Step 3: Remove duplicate ratings per user-book pair
df = df.drop_duplicates(subset=['title', 'user'])

# Step 4: Create pivot table (books as rows, users as columns)
ratings_pivot = df.pivot(index='title', columns='user', values='rating').fillna(0)
ratings_matrix = ratings_pivot.values

# Step 5: Train the k-NN model
model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(ratings_matrix)

# Step 6: Build index mapping
book_titles = ratings_pivot.index
indices = pd.Series(range(len(book_titles)), index=book_titles)

# Step 7: Define recommendation function
def get_recommends(book = ""):
  x=ratings_pivot.loc[book].array.reshape(1, -1)
  distances,indices=model.kneighbors(x,n_neighbors=6)
  R_books=[]
  for distance,indice in zip(distances[0],indices[0]):
    if distance!=0:
      R_book=ratings_pivot.index[indice]
      R_books.append([R_book,distance])
  recommended_books=[book,R_books[::-1]]
  return recommended_books

In [144]:
get_recommends('The Queen of the Damned (Vampire Chronicles (Paperback))')


['The Queen of the Damned (Vampire Chronicles (Paperback))',
 [['Catch 22', 0.7939835419270879],
  ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448657003312193],
  ['Interview with the Vampire', 0.7345068863988313],
  ['The Tale of the Body Thief (Vampire Chronicles (Paperback))',
   0.5376338446489461],
  ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178411864186412],
  ['The Queen of the Damned (Vampire Chronicles (Paperback))',
   1.1102230246251565e-16]]]

In [145]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016210581447822], ['The Weight of Water', 0.7708583572697412], ['The Surgeon', 0.7699410973804288], ['I Know This Much Is True', 0.7677075092617776], ['The Lovely Bones: A Novel', 0.7234864549790632], ["Where the Heart Is (Oprah's Book Club (Paperback))", 2.220446049250313e-16]]]
You passed the challenge! 🎉🎉🎉🎉🎉
